In [1]:
import json

In [3]:
fndir = '../resource/v0.8/'

In [4]:
def load_anno():
    with open(fndir+'KFN_annotations.json','r') as f:
        kfn_annos = json.load(f)
    return kfn_annos
kfn_annos = load_anno()

In [5]:
def load_kfn_lus():
    with open('../resource/v0.8/KFN_lus.json','r') as f:
        kfn_lus = json.load(f)
    return kfn_lus
kfn_lus = load_kfn_lus()

In [42]:
# 1) unpublishing all LUS
for i in kfn_lus:
    # 1) unpublishing all LUS
    publish = {}
    publish['is_publish'] = False
    publish['version'] = '0.8'
    i['publish'] = publish
    
    # 2) empty sejong mapping and all annotation_ids
    i['mapSejong'] = False
    i['sejong_annotation_id'] = []
    i['ko_annotation_id'] = []
    
    # 3) empty surface forms
    i['surface_forms'] = []
    
    # 4) rev POS
    pos = i['lu'].split('.')[1]
    i['pos'] = pos
    
    # 5) separate entry and lu
    word = i['lu'].split('.')[0]
    entry = word+'.'+pos
    i['lexical_entry'] = entry
    
    # etc
    enlus = i['en_lu']
    for enlu in enlus:
        enlu = enlu.lower()
    enlus = list(set(enlus))
    i['en_lu'] = enlus
    
    i['fe_list'] = []
    
    
print('before:')
print(kfn_lus[1])

before:
{'fe_list': [], 'fid': 195, 'surface_forms': [], 'mapSejong': False, 'frameName': 'Political_locales', 'publish': {'is_publish': False, 'version': '0.8'}, 'lu_id': 2, 'lexeme': '나라', 'ko_annotation_id': [], 'lu': '나라.n.Political_locales', 'lemma_var': ['나라'], 'sejong_annotation_id': [], 'pos': 'n', 'en_lu': ['kuni .国 ', 'National', 'country'], 'lexical_entry': '나라.n'}


In [43]:
# 2) rev ko_annotation_id
for i in kfn_annos:
    sent_id = i['text']['sent_id']
    for koanno in i['frameAnnotation']['ko_annotations']:
        anno_lu = koanno['lu']
        ko_annotation_id = koanno['ko_annotation_id']
        surface_form = ''
        anno_fe_list = []
        for deno in koanno['denotations']:
            if deno['role'] == 'TARGET':
                surface_form = deno['text']
            else:
                fe = deno['obj']
                anno_fe_list.append(fe)
        add = False
        for kfn_lu in kfn_lus:
            if anno_lu == kfn_lu['lu']:
                ko_anno_ids = kfn_lu['ko_annotation_id']
                ko_anno_ids.append(ko_annotation_id)
                kfn_lu['ko_annotation_id'] = ko_anno_ids
                
                publish = kfn_lu['publish']
                publish['is_publish'] = True
                publish['version'] = '0.8'
                kfn_lu['publish'] = publish
                
                if surface_form:
                    surface_forms = kfn_lu['surface_forms']
                    surface_forms.append(surface_form)
                    surface_forms = list(set(surface_forms))
                    kfn_lu['surface_forms'] = surface_forms
                    
                lu_fe_list = kfn_lu['fe_list']
                lu_fe_list = lu_fe_list + anno_fe_list
                lu_fe_list = list(set(lu_fe_list))
                kfn_lu['fe_list'] = lu_fe_list
                    
                    
                add = True
        if add == False:
            print('ERROR', sent_id)

print('after:')
print(kfn_lus[1])

after:
{'fe_list': ['Locale', 'Descriptor'], 'fid': 195, 'surface_forms': ['나라입니다.', '나라의', '나라에서,', '나라에서', '나라는', '나라가', '나라로의', '나라도', '나라와의', '나라들이', '나라를', '「나라를', '나라에'], 'mapSejong': False, 'frameName': 'Political_locales', 'publish': {'is_publish': True, 'version': '0.8'}, 'lu_id': 2, 'lexeme': '나라', 'ko_annotation_id': [222, 2373, 2379, 3373, 3384, 3770, 3826, 4452, 5874, 6686, 6703, 6726, 6863, 8526, 9035, 9179, 9180, 10524, 12036, 13746, 13757, 13940], 'lu': '나라.n.Political_locales', 'lemma_var': ['나라'], 'sejong_annotation_id': [], 'pos': 'n', 'en_lu': ['kuni .国 ', 'National', 'country'], 'lexical_entry': '나라.n'}


In [44]:
# 3) rev files for new LU ids
rev_lus = []
for lu in kfn_lus:
    is_publish = lu['publish']['is_publish']
    if is_publish == True:
        rev_lus.append(lu)
lu_id = 1
for lu in rev_lus:
    lu['lu_id'] = lu_id
    lu_id += 1

print(rev_lus[-1])
print(len(rev_lus))

{'fe_list': ['Depictive', 'Speaker', 'Message', 'Time'], 'fid': 43, 'surface_forms': ['동안', '말을', '말도'], 'mapSejong': False, 'frameName': 'Statement', 'publish': {'is_publish': True, 'version': '0.8'}, 'lu_id': 5663, 'lexeme': '말', 'ko_annotation_id': [8026, 11630, 11937], 'lu': '말.n.Statement', 'lemma_var': ['말'], 'sejong_annotation_id': [], 'pos': 'n', 'en_lu': ['kotoba .言葉 ', 'statement', 'iu .言う '], 'lexical_entry': '말.n'}
5663


In [45]:
# 4) save data
with open(fndir+'KFN_lus.json','w') as f:
    json.dump(rev_lus, f, ensure_ascii=False, indent=4)